In [ ]:
pip install ijson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Instructions:
The next few cells are data-preprocessing. Skip to the "LOAD processed dataset to start running the code


# Create Dataframe Files for Processing
## **(DO NOT RUN THIS UNLESS YOU NEED THE FILES)**

In [ ]:
import pickle
import json

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'
# NOTE: THIS MAY HAVE TO BE RAN LOCALLY DUE TO MEMORY CONSTRAINTS
with open(DATA_PATH, 'rb') as f:
    print('Loading all data...')
    data_json = json.load(f)
    print('Loaded all data.')

df_data = pd.json_normalize(data_json)

SEPARATE_FILES = 10

data_len = len(df_data)
amount = data_len/SEPARATE_FILES
for i in range(0, SEPARATE_FILES):
    print('Creating separate file df_data_%i.pickle' % i)
    with open('df_data_%i.pickle' % i, 'wb') as f:
        start = i*amount
        end = (i+1)*amount
        if i == SEPARATE_FILES-1:
            end = data_len
        pickle.dump(df_data.iloc[start:end])

Loading all data...


# Data Cleaning and Formatting

## Load Subset DataFrame

In [ ]:
import pickle
import pandas as pd

#SUBSET_NUMBER = 0 # choose between 0 to SEPARATE_FILES
#SUBSET_PATH = 'df_data_%i.pickle' % SUBSET_NUMBER
SUBSET_PATHS = [
    '/content/drive/MyDrive/Twibot-20/dataframes/df_data_0.pickle',
    # '/content/drive/MyDrive/Twibot-20/dataframes/df_data_1.pickle',
    # '/content/drive/MyDrive/Twibot-20/dataframes/df_data_2.pickle'
]

df_data = pd.DataFrame()
for SUBSET_PATH in SUBSET_PATHS:
  with open(SUBSET_PATH, 'rb') as f:
      df_sub = pickle.load(f)
  df_data = pd.concat([df_data, df_sub])

df_data

,created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,withheld,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,text
0,Tue Nov 18 10:27:25 +0000 2008,"VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",{'url': {'urls': [{'url': 'https://t.co/7hsiK8...,u17461978,"Orlando, FL",SHAQ,NaN,http://pbs.twimg.com/profile_images/1673907275...,False,https://t.co/7hsiK8cCKW,SHAQ,True,NaN,15349596.0,692.0,9798.0,45568.0,NaN
1,Sun Aug 23 07:35:11 +0000 2020,,{'description': {'urls': []}},u1297437077403885568,,Jennifer Fishpaw,NaN,http://pbs.twimg.com/profile_images/1297437406...,False,None,JenniferFishpaw,False,NaN,0.0,44.0,0.0,0.0,NaN
2,Thu Nov 27 18:47:32 +0000 2008,Owner @ Parscale Strategy. Senior Advisor Digi...,{'url': {'urls': [{'url': 'https://t.co/GooZcY...,u17685258,Florida,Brad Parscale,NaN,http://pbs.twimg.com/profile_images/1295453225...,False,https://t.co/GooZcYDqFg,parscale,True,NaN,762839.0,475.0,5518.0,3201.0,NaN
3,Wed Aug 06 15:12:10 +0000 2008,Bringing you the important stuff like breaking...,{'url': {'urls': [{'url': 'https://t.co/RtP9QY...,u15750898,"Tampa, FL",FOX 13 Tampa Bay,NaN,http://pbs.twimg.com/profile_images/1293193013...,False,https://t.co/RtP9QYEZCq,FOX13News,True,NaN,327587.0,4801.0,192876.0,1744.0,NaN
4,Sat Aug 10 03:25:35 +0000 2013,MOTIVATION 3 OUT NOW 🔥 Singles: ‘Lil Shawdy’ &...,{'url': {'urls': [{'url': 'https://t.co/5cY8GW...,u1659167666,"Jacksonville Beach, FL",Vonte The Plug 🎤🔌,NaN,http://pbs.twimg.com/profile_images/1181662400...,False,https://t.co/5cY8GWvk8E,VonteThePlugNC,False,NaN,13324.0,647.0,103.0,44.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2247846,NaN,NaN,NaN,t2018266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@GhettoGeasy @ledrizlla Heyy kur mundemi me e ...
2247847,NaN,NaN,NaN,t2018267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,La fuerte corriente de cizalla deja al descubi...
2247848,NaN,NaN,NaN,t2018268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Amplia región de Nubes de tormenta en Oaxaca y...
2247849,NaN,NaN,NaN,t2018269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dos zonas de perturbación en el Pacífico con e...


## Only Select Users and get User Specific Features

In [ ]:
df_user = df_data[['id', 
                  'public_metrics.followers_count', 
                  'public_metrics.following_count',
                  'public_metrics.tweet_count',
                  'public_metrics.listed_count', 
                  'verified']]

# convert bool to int
df_user['verified'] = df_user['verified'].replace({'True ': 1, 'False ': 0}) # this is slow

df_user = df_user.loc[df_user['id'].str.contains('u')]
df_user.set_index('id', inplace=True)
df_user

<ipython-input-2-0a87b99f3d91>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['verified'] = df_user['verified'].replace({'True ': 1, 'False ': 0}) # this is slow


,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,verified
id,,,,,
u17461978,15349596.0,692.0,9798.0,45568.0,1.0
u1297437077403885568,0.0,44.0,0.0,0.0,0.0
u17685258,762839.0,475.0,5518.0,3201.0,1.0
u15750898,327587.0,4801.0,192876.0,1744.0,1.0
u1659167666,13324.0,647.0,103.0,44.0,0.0
...,...,...,...,...,...
u470607493,47.0,583.0,0.0,0.0,0.0
u171175886,1463.0,1295.0,8057.0,33.0,0.0
u31429209,129.0,784.0,430.0,0.0,0.0


## Add Bot Label to Data

In [ ]:
# Load label data
LABEL_PATH = '/content/drive/MyDrive/Twibot-20/label.csv'
df_label = pd.read_csv(LABEL_PATH)
df_label.set_index('id', inplace=True)
df_label

,label
id,
u17461978,human
u1297437077403885568,bot
u17685258,human
u15750898,human
u1659167666,bot
...,...
u452754350,bot
u850435801687183362,bot
u2188795745,bot


In [ ]:
# merge label data to df_user
df_user['label'] = df_label['label']

# drop users that aren't labeled in dataset
df_user.dropna(inplace=True)
df_user

,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,verified,label
id,,,,,,
u17461978,15349596.0,692.0,9798.0,45568.0,1.0,human
u1297437077403885568,0.0,44.0,0.0,0.0,0.0,bot
u17685258,762839.0,475.0,5518.0,3201.0,1.0,human
u15750898,327587.0,4801.0,192876.0,1744.0,1.0,human
u1659167666,13324.0,647.0,103.0,44.0,0.0,bot
...,...,...,...,...,...,...
u452754350,7760.0,8104.0,2330.0,106.0,0.0,bot
u850435801687183362,8446.0,408.0,50168.0,55.0,0.0,bot
u2188795745,309.0,1961.0,2950.0,3.0,0.0,bot


In [ ]:
with open('df_user.pickle', 'wb') as f:
    # Use the pickle.dump() function to save the dictionary to the file
    pickle.dump(df_user, f)

## Tweet Level Feature Extraction
Do not run unless changing stuff - use loaded pickle file instead

Helper Functions

In [ ]:
def num_links(tweet_text: str) -> int:
    return len(re.findall(r'https?://\S+', tweet_text))

def num_hashtags(tweet_text: str) -> int:
    return len(re.findall(r'#\S+', tweet_text))

def num_characters(tweet_text: str) -> int:
    return len(tweet_text)

def num_words(tweet_text: str) -> int:
    return len(tweet_text.split(' '))
    
def num_mentions(tweet_text: str) -> int:
    return len(re.findall(r'@\S+', tweet_text))

def is_retweet(tweet_text: str) ->int:
    return int(tweet_text[0:2] == "RT")

Process all tweets and gather data

In [ ]:
user_dict = {}
edge_path = '/content/drive/MyDrive/Twibot-20/edge.csv'
edge_file = pd.read_csv(edge_path, index_col = "target_id")
edge_file = edge_file[edge_file['relation'] == 'post']

user_dict = {}

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'
with open(DATA_PATH, 'rb') as f:
    for record in ijson.items(f, "item"):
      #process each record
      if "t" in record["id"]:
        user = edge_file.loc[record["id"], 'source_id']
        if user not in user_dict.keys():
          user_dict[user] = {"Number": 0, "Links": [], "Hashtags": [], "Mentions": [], "Words": [], "RT": 0}
        if user_dict[user]["Number"] < 50:
          user_dict[user]["Number"] += 1
          user_dict[user]["Links"].append(num_links(record["text"]))
          user_dict[user]["Hashtags"].append(num_hashtags(record["text"]))
          user_dict[user]["Mentions"].append(num_mentions(record["text"]))
          user_dict[user]["Words"].append(num_words(record["text"]))
          user_dict[user]["RT"] += is_retweet(record["text"])

import pickle

# Open a file in binary write mode
with open('user_dict.pickle', 'wb') as f:
    # Use the pickle.dump() function to save the dictionary to the file
    pickle.dump(user_dict, f)

Load tweet file and process it to calculate means and std to use as features. 

---



---



---



---



---



---



---



---



---



In [ ]:
# Load tweet pickle
Tweet_path = '/content/drive/MyDrive/Twibot-20/dataframes/user_dict.pickle'
user_dict = pd.read_pickle(Tweet_path)

In [ ]:
import numpy as np
def calculate_mean_std(numbers):
    """
    Takes in a list of numbers and calculates their mean and standard deviation
    """
    # Convert the list to a NumPy array
    numbers = np.array(numbers)
    
    # Calculate the mean and standard deviation
    mean = np.mean(numbers)
    std = np.std(numbers)
    
    # Return the results
    return mean, std

In [ ]:
for user in user_dict.keys():
  user_dict[user]['Links_mean'], user_dict[user]['Links_std'] = calculate_mean_std(user_dict[user]['Links'])
  user_dict[user]['Hashtags_mean'], user_dict[user]['Hashtags_std'] = calculate_mean_std(user_dict[user]['Hashtags'])
  user_dict[user]['Mentions_mean'], user_dict[user]['Mentions_std'] = calculate_mean_std(user_dict[user]['Mentions'])
  user_dict[user]['Words_mean'], user_dict[user]['Words_std'] = calculate_mean_std(user_dict[user]['Words'])
  del user_dict[user]['Links']
  del user_dict[user]['Hashtags']
  del user_dict[user]['Mentions']
  del user_dict[user]['Words']
  del user_dict[user]['Number']

In [ ]:
import pickle
with open('user_dict_std.pickle', 'wb') as f:
    # Use the pickle.dump() function to save the dictionary to the file
    pickle.dump(user_dict, f)

## Combine Tweet Features with users

In [ ]:
user_tweet_dict_path = '/content/drive/MyDrive/Twibot-20/dataframes/user_dict_std.pickle'
df_user_path = '/content/drive/MyDrive/Twibot-20/dataframes/df_user.pickle'

user_tweet_dict = pd.read_pickle(user_tweet_dict_path)
df_user = pd.read_pickle(df_user_path)

In [ ]:
pd_dict = pd.DataFrame.from_dict(user_tweet_dict, orient='index')

merged_df = df_user.merge(pd_dict, left_index=True, right_index=True, how='inner')

print(len(merged_df))
# Display the merged dataframe
print(merged_df)


11746
                     public_metrics.followers_count  \
u17461978                                15349596.0   
u17685258                                  762839.0   
u15750898                                  327587.0   
u1659167666                                 13324.0   
u34743251                                12601567.0   
...                                             ...   
u452754350                                   7760.0   
u850435801687183362                          8446.0   
u2188795745                                   309.0   
u940687680                                    154.0   
u3385331674                                    68.0   

                     public_metrics.following_count  \
u17461978                                     692.0   
u17685258                                     475.0   
u15750898                                    4801.0   
u1659167666                                   647.0   
u34743251                                      96.0   
...

In [ ]:
#Final Feature List
print(merged_df.columns)

Index(['public_metrics.followers_count', 'public_metrics.following_count',
       'public_metrics.tweet_count', 'public_metrics.listed_count', 'verified',
       'label', 'Number', 'RT', 'Links_mean', 'Links_std', 'Hashtags_mean',
       'Hashtags_std', 'Mentions_mean', 'Mentions_std', 'Words_mean',
       'Words_std'],
      dtype='object')


In [ ]:
with open('processed_dataset.pickle', 'wb') as f:
    # Use the pickle.dump() function to save the dictionary to the file
    pickle.dump(merged_df, f)

#LOAD Processed Dataset

In [ ]:
df_user_path = '/content/drive/MyDrive/Twibot-20/dataframes/processed_dataset.pickle'
df_user = pd.read_pickle(df_user_path)

# Use Random Forest to Predict Human or Bot

## Split train and test

In [ ]:
TRAIN_SPLIT = 0.7
stop_point = int(len(df_user)*TRAIN_SPLIT)
df_train = df_user.iloc[:stop_point]
df_test = df_user.iloc[stop_point:]

df_train.head()

,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,verified,label
id,,,,,,
u17461978,15349596.0,692.0,9798.0,45568.0,1.0,human
u1297437077403885568,0.0,44.0,0.0,0.0,0.0,bot
u17685258,762839.0,475.0,5518.0,3201.0,1.0,human
u15750898,327587.0,4801.0,192876.0,1744.0,1.0,human
u1659167666,13324.0,647.0,103.0,44.0,0.0,bot


## Function to Train on Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

def train_and_pred(model):
  print('Training...')
  # fit model
  model.fit(df_train.drop(['label'], axis=1), df_train['label'])

  test_pred = model.predict(df_test.drop('label', axis=1))

  # get accuracy
  accuracy = accuracy_score(test_pred, df_test['label'])
  print('accuracy: %f' % accuracy)
  return accuracy

## Train and test here for accuracy

In [ ]:
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'MLPClassifier': MLPClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier()
}
accuracies = {model: 0 for model in models}
for model in models:
  accuracies[model] = train_and_pred(models[model])

# testing_estimators = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# accuracies = {estimator: 0 for estimator in testing_estimators}
# for estimators in testing_estimators:
#   model = AdaBoostClassifier(n_estimators=estimators)
#   accuracies[estimators] = train_and_pred(model)
accuracies

Training...
accuracy: 0.791995
Training...
accuracy: 0.804679
Training...
accuracy: 0.724634
Training...
accuracy: 0.571308
Training...
accuracy: 0.707159


{'RandomForestClassifier': 0.7919954904171365,
 'AdaBoostClassifier': 0.8046786922209695,
 'DecisionTreeClassifier': 0.7246335963923337,
 'MLPClassifier': 0.5713077790304397,
 'KNeighborsClassifier': 0.7071589627959414}

### Plot results of training

In [ ]:
import plotly.express as px
df_acc = pd.DataFrame(accuracies.items())
df_acc.columns = ['Models', 'Accuracy']

fig = px.bar(df_acc,x = df_acc.columns[0], y = df_acc.columns[1])
fig.show()

In [ ]:
def num_links(tweet_text: str) -> int:
    return len(re.findall(r'https?://\S+', tweet_text))

def num_hashtags(tweet_text: str) -> int:
    return len(re.findall(r'#\S+', tweet_text))

def num_characters(tweet_text: str) -> int:
    return len(tweet_text)

def num_words(tweet_text: str) -> int:
    return len(tweet_text.split(' '))
    
def num_mentions(tweet_text: str) -> int:
    return len(re.findall(r'@\S+', tweet_text))

def is_retweet(tweet_text: str) ->int:
    return int(tweet_text[0:2] == "RT")

#TODO: we may need to perform this feature later, after we have linked all tweets of the same user
def edit_distance(a: str, b: str) -> int:
    #return edit_distance(a.lower(), b.lower())
    return 0

In [ ]:
import pandas as pd
import json
import ijson
import numpy as np
import re

#PULLING TWEETS FROM node.json

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'
raw_tweets = pd.DataFrame(columns=["Tweet_ID", "Tweet_Text", "Characters", "Words", "Mentions", "Links", "Hashtags", "EditDist"])

#Pull tweets total 
total = 10000000000000000000000
#Process 20 tweets at once
i = 20
count = 0

blockDict = {"Tweet_ID": [], "Tweet_Text": [], "Characters": [], "Words": [], "Mentions": [], "Links": [], "Hashtags": [], "EditDist": []}
tweet_ID = []
tweet_text = []
link_list = []
hash_list = []
dist_list = []
char_list = []
word_list = []
mention_list = []

with open(DATA_PATH, 'rb') as f:
    for record in ijson.items(f, "item"):
      #process each record
      if "t" in record["id"]:
        #print(record)
        # Appending to dictionary #
        tweet_ID.append(record["id"])
        tweet_text.append(record["text"])
        # ============ ADD FEATURES AS ENTRIES BELOW ============ #
        link_list.append(num_links(record["text"]))
        hash_list.append(num_hashtags(record["text"]))
        dist_list.append(0) # not implemented yet
        char_list.append(num_characters(record["text"]))
        word_list.append(num_words(record["text"]))
        mention_list.append(num_mentions(record["text"]))

        count += 1
      if count == i:
        blockDict["Tweet_ID"] = tweet_ID
        blockDict["Tweet_Text"] = tweet_text
        blockDict["Links"] = link_list
        blockDict["Hashtags"] = hash_list
        blockDict["EditDist"] = dist_list
        blockDict["Characters"] = char_list
        blockDict["Words"] = word_list
        blockDict["Mentions"] = mention_list
        
        raw_tweets = pd.concat([raw_tweets, pd.DataFrame(blockDict)], ignore_index=True)
        
        tweet_ID.clear()
        tweet_text.clear()
        link_list.clear()
        hash_list.clear()
        dist_list.clear()
        char_list.clear()
        word_list.clear()
        mention_list.clear()

      if count == total:
        break

raw_tweets.set_index("Tweet_ID", inplace=True)
raw_tweets.head()

with open('tweet_features.pickle' % i, 'wb') as f:
  pickle.dump(raw_tweets)
      


,Tweet_ID,Tweet_Text,Links,Hashtags,EditDist
0,t0,RT @CarnivalCruise: 🎉 Are you ready to see wha...,0.0,0.0,0.0
1,t1,Who has time for receipts? Not me. @epson rece...,2.0,0.0,0.0
2,t2,Steady wants to encourage you to invest in you...,2.0,1.0,0.0
3,t3,"Good one, @rishid. But let’s see if y'all can ...",2.0,2.0,0.0
4,t4,#lsunationalchamps\n,0.0,1.0,0.0


In [ ]:
# COMPUTE EDIT DISTANCE

import csv
DATA_PATH = "edge_output2.csv"
# CSV FORMAT: SOURCE_ID, 

with open(DATA_PATH, 'r') as file:
    edges = pd.read_csv(file)

edges['target_id'] = edges['target_id'].apply(lambda x: x.strip('\'][\'').split('\', \''))
edges.set_index("source_id", inplace=True)
tweet_edges = edges[edges['relation'] == 'post'] # drops all user to user edges

#block length 
n = 20

# Looping through all users, index should be the userID
for index, row in tweet_edges.iterrows():
    tids = row['target_id']
    
    
    tweets = []
    avg_dist = 0
    for i, _ in enumerate(tids): #initialize rolling tweets
        if i == n:
            break
        tweets.append[raw_tweets.at[tids[i], 'Tweet_Text']]
        

    for i in range(len(tids)):
        
        # ============= EDIT DISTANCE ============= #
        start = 0
        if i > len(tids) - n:
            start = i - (len(test_list) - n if len(test_list) - n > 0 else 0)
    
        for j _ in enumerate(tweets):
            if j == start:
                continue
            avg_dist += edit_distance(tweets[start], tweets[j])
        raw_tweets.at[tids[i], "EditDist"] = avg_dist/(len(tweets)-1)
        avg_dist = 0
        if i < len(tids) - n:
            tweets.pop(0)
            tweets.append(raw_tweets.at[tids[i+n], 'Tweet_Text'])
        # ============= EDIT DISTANCE END ============= #
        
        # ============= COMPUTE AVG TWEET STUFF HERE ============= #


# Playground Code

In [ ]:
import pandas as pd
import json
import ijson
import numpy as np
import re

In [ ]:
user_dict = {}
edge_path = '/content/drive/MyDrive/Twibot-20/edge.csv'
edge_file = pd.read_csv(edge_path, index_col = "target_id")
edge_file = edge_file[edge_file['relation'] == 'post']

user_dict = {}

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'
with open(DATA_PATH, 'rb') as f:
    for record in ijson.items(f, "item"):
      #process each record
      if "t" in record["id"]:
        user = edge_file.loc[record["id"], 'source_id']
        if user not in user_dict.keys():
          user_dict[user] = {"Number": 0, "Links": [], "Hashtags": [], "Mentions": [], "Words": [], "RT": 0}
        if user_dict[user]["Number"] < 50:
          user_dict[user]["Number"] += 1
          user_dict[user]["Links"].append(num_links(record["text"]))
          user_dict[user]["Hashtags"].append(num_hashtags(record["text"]))
          user_dict[user]["Mentions"].append(num_mentions(record["text"]))
          user_dict[user]["Words"].append(num_words(record["text"]))
          user_dict[user]["RT"] += is_retweet(record["text"])

import pickle

# Open a file in binary write mode
with open('user_dict.pickle', 'wb') as f:
    # Use the pickle.dump() function to save the dictionary to the file
    pickle.dump(user_dict, f)

In [ ]:
edge_file = edge_file[edge_file['relation'] == 'post']
print(edge_file)

           source_id relation
target_id                    
t0         u17461978     post
t1         u17461978     post
t2         u17461978     post
t3         u17461978     post
t4         u17461978     post
...              ...      ...
t33488187  u16622009     post
t33488188  u16622009     post
t33488189  u16622009     post
t33488190  u16622009     post
t33488191  u16622009     post

[33488192 rows x 2 columns]


In [ ]:
user_dict = {}

In [ ]:
#Extra 
grouped_df = edge_file.groupby('source_id')
print(len(grouped_df))

206043


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'
with open(DATA_PATH, 'rb') as f:
    for record in ijson.items(f, "item"):
      #process each record
      if "t" in record["id"]:
        user = edge_file.loc[record["id"], 'source_id']
        if user not in user_dict.keys():
          user_dict[user] = {"Number": 0, "Links": [], "Hashtags": [], "Mentions": [], "Words": [], "RT": 0}
        if user_dict[user]["Number"] < 50:
          user_dict[user]["Number"] += 1
          user_dict[user]["Links"].append(num_links(record["text"]))
          user_dict[user]["Hashtags"].append(num_hashtags(record["text"]))
          user_dict[user]["Mentions"].append(num_mentions(record["text"]))
          user_dict[user]["Words"].append(num_words(record["text"]))
          user_dict[user]["RT"] += is_retweet(record["text"])


In [ ]:
len(user_dict)


206043

In [ ]:
import pickle

# Open a file in binary write mode
with open('user_dict.pickle', 'wb') as f:
    # Use the pickle.dump() function to save the dictionary to the file
    pickle.dump(user_dict, f)

In [ ]:
import pandas as pd
import json
import ijson
import numpy as np
import re

#PULLING TWEETS FROM node.json

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'
raw_tweets = pd.DataFrame(columns=["Tweet_ID", "Tweet_Text", "Characters", "Words", "Mentions", "Links", "Hashtags", "EditDist"])


######################
#Pull tweets total 
total = 10000000000000000000000
#Process 20 tweets at once
i = 20
count = 0

blockDict = {"Tweet_ID": [], "Tweet_Text": [], "Characters": [], "Words": [], "Mentions": [], "Links": [], "Hashtags": [], "EditDist": []}
tweet_ID = []
tweet_text = []
link_list = []
hash_list = []
dist_list = []
char_list = []
word_list = []
mention_list = []

with open(DATA_PATH, 'rb') as f:
    for record in ijson.items(f, "item"):
      #process each record
      if "t" in record["id"]:
        #print(record)
        # Appending to dictionary #
        tweet_ID.append(record["id"])
        tweet_text.append(record["text"])
        # ============ ADD FEATURES AS ENTRIES BELOW ============ #
        user = edge_file.loc[record["id"], 'source_id']
        if user not in user_dict.keys():
          user_dict[user] = {"Number": 0, "Links": [], "Hashtags": [], "Mentions": [], "Words": [], "RT": 0}
        user_dict[user]["Number"] += 1
        user_dict[user]["Links"].append(num_links(record["text"]))
        user_dict[user]["Hashtags"].append(num_hashtags(record["text"]))
        user_dict[user]["Mentions"].append(num_mentions(record["text"]))
        user_dict[user]["Words"].append(num_words(record["text"]))
        user_dict[user]["RT"] += is_retweet(record["text"])
        
        link_list.append(num_links(record["text"]))
        hash_list.append(num_hashtags(record["text"]))
        dist_list.append(0) # not implemented yet
        char_list.append(num_characters(record["text"]))
        word_list.append(num_words(record["text"]))
        mention_list.append(num_links(record["text"]))

        count += 1
      if count == i:
        blockDict["Tweet_ID"] = tweet_ID
        blockDict["Tweet_Text"] = tweet_text
        blockDict["Links"] = link_list
        blockDict["Hashtags"] = hash_list
        blockDict["EditDist"] = dist_list
        blockDict["Characters"] = char_list
        blockDict["Words"] = word_list
        blockDict["Mentions"] = mention_list
        
        raw_tweets = pd.concat([raw_tweets, pd.DataFrame(blockDict)], ignore_index=True)
        
        tweet_ID.clear()
        tweet_text.clear()
        link_list.clear()
        hash_list.clear()
        dist_list.clear()
        char_list.clear()
        word_list.clear()
        mention_list.clear()

      if count == total:
        break

raw_tweets.set_index("Tweet_ID", inplace=True)
raw_tweets.head()

with open('tweet_features.pickle' % i, 'wb') as f:
  pickle.dump(raw_tweets)
      


In [ ]:
#tweet feature extraction
import pandas as pd
DATA_PATH = '/content/drive/MyDrive/Twibot-20/edge.csv'

data = pd.read_csv(DATA_PATH)
print(data)



# with open(DATA_PATH, 'rb') as f:
    
#     for record in ijson.items(f, "item"):
#       print(record)
#       break


          source_id relation  target_id
0         u17461978     post         t0
1         u17461978     post         t1
2         u17461978     post         t2
3         u17461978     post         t3
4         u17461978     post         t4
...             ...      ...        ...
33716166  u16622009     post  t33488187
33716167  u16622009     post  t33488188
33716168  u16622009     post  t33488189
33716169  u16622009     post  t33488190
33716170  u16622009     post  t33488191

[33716171 rows x 3 columns]


In [ ]:
#User feature extraction
import pandas as pd

DATA_PATH = '/content/drive/MyDrive/output.xlsx'
df_data = pd.read_excel(DATA_PATH)
user_features = df_data[['ID', 'domain', 'profile.screen_name', 'profile.profile_location', 'profile.description', 'profile.followers_count', 'profile.friends_count', 'profile.created_at', 'profile.geo_enabled', 'profile.verified', 'profile.default_profile']]

user_features.head()

,ID,domain,profile.screen_name,profile.profile_location,profile.description,profile.followers_count,profile.friends_count,profile.created_at,profile.geo_enabled,profile.verified,profile.default_profile
0,17461978,"['Politics', 'Bussiness', 'Entertainment']",SHAQ,"{'id': '55b4f9e5c516e0b6', 'url': 'https://api...","VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",15349596,692,Tue Nov 18 10:27:25 +0000 2008,True,True,False
1,1297437077403885568,['Politics'],JenniferFishpaw,None,,0,44,Sun Aug 23 07:35:11 +0000 2020,False,False,True
2,17685258,"['Politics', 'Entertainment', 'Sports']",parscale,None,Owner @ Parscale Strategy. Senior Advisor Digi...,762839,475,Thu Nov 27 18:47:32 +0000 2008,True,True,False
3,15750898,['Politics'],FOX13News,None,Bringing you the important stuff like breaking...,327587,4801,Wed Aug 06 15:12:10 +0000 2008,True,True,False
4,1659167666,['Politics'],VonteThePlugNC,"{'id': '5e281c17a74c170f', 'url': 'https://api...",MOTIVATION 3 OUT NOW 🔥 Singles: ‘Lil Shawdy’ &...,13324,647,Sat Aug 10 03:25:35 +0000 2013,False,False,True


In [ ]:
import pandas as pd
import json
import ijson

#EXAMPLE OF A DATAFRAME FOR FEATURES

DATA_PATH = '/content/drive/MyDrive/Twibot-20/node.json'

#chunksize = 10
i = 0

with open(DATA_PATH, 'rb') as f:
    for record in ijson.items(f, "item"):
      #process each record
      if 't' in record['id']:
        print(record)
        i += 1
      if i > 10:
        break

      
      

print('done')

{'id': 't0', 'text': 'RT @CarnivalCruise: 🎉 Are you ready to see what our newest ship’s name will be? 🎉 Thanks to all our partners for helping us unbox the name.…\n'}
{'id': 't1', 'text': 'Who has time for receipts? Not me. @epson receipt scanners make it easy. No mess = no stress! Check it out at https://t.co/ofqbTdz0Qk. https://t.co/BtYwuyz9N5\n'}
{'id': 't2', 'text': 'Steady wants to encourage you to invest in your financial future. Connect your bank account to @TheSteadyApp and have access to benefits like income insights, online medical visits, and cash grants of up to $1,000! Get started today by visiting https://t.co/5w3AvWd8Q0. #Sponsored https://t.co/ZHBfP5xDMg\n'}
{'id': 't3', 'text': "Good one, @rishid. But let’s see if y'all can do better than that. Come on, everybody. Show me your best #HandShaq! #ad https://t.co/xCloeLRfuM https://t.co/urVSOfTmT2\n"}
{'id': 't4', 'text': '#lsunationalchamps\n'}
{'id': 't5', 'text': 'I stand with the student athletes! #WeWantToPlay\n'}
{'i